In [81]:
from serpapi import GoogleSearch
from dotenv import load_dotenv
import os
import pandas as pd

# pip install python-dotenv
# pip install google-search-results

In [109]:
search_term = 'Data Analyst'
search_location = 'United States'

load_dotenv()

params = {
    #"3b49bf2759555c8a50f74b4cca39c5454f13d6f55a008506210b3b62127531a2"
    "api_key": os.getenv('API_KEY'),
    "device": "desktop",
    "engine": "google_jobs",
    "google_domain": "google.com",
    "q": search_term,
    "hl": "en",
    "gl": "us",
    "location": search_location,
    "chips": "date_posted:today",
    "start": 0
}

Creating the first version of the collecting file

In [ ]:
final_data = pd.DataFrame()

for _ in range(30):
    
    try:
        search = GoogleSearch(params)
        results = search.get_dict()
        jobs_df = results['jobs_results']
    except:
        break
    
    jobs_df = pd.DataFrame(jobs_df)

    jobs_df = pd.concat([pd.DataFrame(jobs_df),
        pd.json_normalize(jobs_df['detected_extensions'])],
        axis=1).drop('detected_extensions', axis=1)

    final_data = pd.concat([final_data, jobs_df], ignore_index=True)

    params["start"] = params["start"] + 10

    print(params["start"])

#final_data.to_csv('data_analyst_jobs.csv')


Upload Data to BigQuery

In [95]:
# pip install google-cloud-bigquery

from google.cloud import bigquery
from google.oauth2 import service_account

In [150]:
load_dotenv()

key_path = os.getenv('ACCOUNT_LOCATION_NOTEBOOK2')

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/bigquery"]
)

upload_data = final_data[['title','company_name','via','description','location','thumbnail','job_id','posted_at','schedule_type','salary','work_from_home']]

client = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id
)

table_id = 'polar-arbor-371523.data_analyst_search.job_list'

table = client.get_table(table_id)

errors = client.insert_rows_from_dataframe(table, upload_data)

if errors == [[]]:
    print('Data Loaded')
else:
    print(errors)

Data Loaded
